In [1]:
import pandas as pd
import requests
import time
import geopandas as gpd
from dateutil.relativedelta import relativedelta
import datetime as dt
import math

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [3]:
# Read in new file
file_loc_raw = "../datasets/resale_hdb_price_raw_13jan.csv"
df_raw = pd.read_csv(file_loc_raw, parse_dates=["month", "lease_commence_date"])

df_raw["floor_area_sqft"] = df_raw["floor_area_sqm"] * 10.7639
df_raw["price_per_sqft"] = (
    df_raw["resale_price"] / df_raw["floor_area_sqft"]
)

# df_raw = df_raw.drop(["remaining_lease", "lease_commence_date"], axis=1)

In [4]:
df_raw.head(1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,473.6116,489.852867


In [ ]:
# df_raw = df_raw.query("month != '2025-01-01'")

In [13]:
df_raw.shape

(197008, 13)

### Begin the synchronisation process

#### 1. Open the masterlist of HDB addresses. This will be used to perform a left join to the new data

In [15]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc, index_col=0, dtype={"postal": "object"}, parse_dates=["lease_commence_date"]
)

masterlist_df.head()

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254
1,ANG MO KIO,207,ANG MO KIO AVE 1,207,ANG MO KIO AVENUE 1,ANG MO KIO 22,560207,207 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29060.485578,38651.052977,1.365821,103.842848,Ang Mo Kio,874.305666,MRT,Red,9084.169820,ANG MO KIO PRIMARY SCHOOL,527.275749
2,ANG MO KIO,208,ANG MO KIO AVE 1,208,ANG MO KIO AVENUE 1,ANG MO KIO 22,560208,208 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-07-01,ANG MO KIO,NORTH-EAST REGION,29045.715075,38609.483079,1.365445,103.842715,Ang Mo Kio,908.966103,MRT,Red,9044.410980,ANG MO KIO PRIMARY SCHOOL,549.157654
3,ANG MO KIO,215,ANG MO KIO AVE 1,215,ANG MO KIO AVENUE 1,ANG MO KIO 22,560215,215 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28924.303291,38732.591142,1.366558,103.841624,Mayflower,781.530965,MRT,Brown,9180.500545,ANG MO KIO PRIMARY SCHOOL,377.594832
4,ANG MO KIO,216,ANG MO KIO AVE 1,216,ANG MO KIO AVENUE 1,ANG MO KIO 22,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...,1976-04-01,ANG MO KIO,NORTH-EAST REGION,28911.052240,38692.616791,1.366197,103.841505,Mayflower,800.632187,MRT,Brown,9142.409400,ANG MO KIO PRIMARY SCHOOL,403.611794


#### 1a: First, check if there are any new addresses that appear in the new dataframe

In [17]:
df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge
186180,2024-12-01,QUEENSTOWN,3 ROOM,79,DAWSON RD,22 TO 24,62.0,Model A,2020-01-01,95 years,766000.0,667.3618,1147.803186,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
187186,2024-12-01,SEMBAWANG,4 ROOM,126C,CANBERRA ST,07 TO 09,93.0,Model A,2020-01-01,95 years,695000.0,1001.0427,694.276078,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
187598,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,116.0,Improved,2020-01-01,95 years 01 month,755000.0,1248.6124,604.671233,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
187600,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,113.0,Improved,2020-01-01,95 years 01 month,745000.0,1216.3207,612.502936,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
187841,2024-12-01,SENGKANG,2 ROOM,456B,SENGKANG WEST RD,04 TO 06,48.0,2-room,2020-01-01,94 years 10 months,400000.0,516.6672,774.192749,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191072,2024-12-01,TAMPINES,4 ROOM,613B,TAMPINES NTH DR 1,07 TO 09,93.0,Model A,2020-01-01,94 years 11 months,750000.0,1001.0427,749.218790,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
191073,2024-12-01,TAMPINES,4 ROOM,608B,TAMPINES NTH DR 1,10 TO 12,93.0,Model A,2020-01-01,94 years 11 months,792000.0,1001.0427,791.175042,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
191074,2024-12-01,TAMPINES,4 ROOM,608B,TAMPINES NTH DR 1,04 TO 06,93.0,Model A,2020-01-01,94 years 11 months,705000.0,1001.0427,704.265662,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
191841,2024-12-01,TAMPINES,5 ROOM,608B,TAMPINES NTH DR 1,04 TO 06,113.0,Improved,2020-01-01,94 years 11 months,852000.0,1216.3207,700.473156,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


#### 1b: If there are new addresses, use the following code to select the correct address, and insert ancillary information like floor area in square feet, price per square foot, and recalculating lease commence date

In [18]:
# First, check for any missing values in the masterlist
missing_val = masterlist_df[masterlist_df.isna().any(axis=1)]
missing_val

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters


In [19]:
# Obtain the index of data points whose block + street name + postal + building information aren't present in the masterlist
for_editing_index = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
).query("_merge != 'both'").drop_duplicates(
    subset=["town", "block", "street_name"]
).index

for_editing = df_raw.loc[for_editing_index, :]
for_editing.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft
186180,2024-12-01,QUEENSTOWN,3 ROOM,79,DAWSON RD,22 TO 24,62.0,Model A,2020-01-01,95 years,766000.0,667.3618,1147.803186
187186,2024-12-01,SEMBAWANG,4 ROOM,126C,CANBERRA ST,07 TO 09,93.0,Model A,2020-01-01,95 years,695000.0,1001.0427,694.276078
187598,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,116.0,Improved,2020-01-01,95 years 01 month,755000.0,1248.6124,604.671233
187841,2024-12-01,SENGKANG,2 ROOM,456B,SENGKANG WEST RD,04 TO 06,48.0,2-room,2020-01-01,94 years 10 months,400000.0,516.6672,774.192749
187860,2024-12-01,SENGKANG,3 ROOM,353B,ANCHORVALE LANE,16 TO 18,68.0,Model A,2021-01-01,95 years 04 months,570000.0,731.9452,778.746824


In [20]:
def obtain_lease_yearmth(row):
    today = pd.to_datetime(dt.date.today().replace(day=1))
    lease_commence = today - pd.DateOffset(
        years=row["lease_year"] + 99, months=row["lease_month"]
    )
    return lease_commence


for_editing["lease_year"] = for_editing["remaining_lease"].str.slice(0, 2).astype("int")
for_editing["lease_month"] = pd.to_numeric(
    for_editing["remaining_lease"].str.slice(9, 11), errors="coerce"
).fillna(0)

for_editing["lease_commence_date"] = for_editing.apply(obtain_lease_yearmth, axis=1)
for_editing = for_editing.drop(
    ["remaining_lease", "lease_year", "lease_month"], axis=1
)

In [21]:
for_editing.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft
186180,2024-12-01,QUEENSTOWN,3 ROOM,79,DAWSON RD,22 TO 24,62.0,Model A,1831-01-01,766000.0,667.3618,1147.803186
187186,2024-12-01,SEMBAWANG,4 ROOM,126C,CANBERRA ST,07 TO 09,93.0,Model A,1831-01-01,695000.0,1001.0427,694.276078
187598,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,116.0,Improved,1830-12-01,755000.0,1248.6124,604.671233
187841,2024-12-01,SENGKANG,2 ROOM,456B,SENGKANG WEST RD,04 TO 06,48.0,2-room,1831-03-01,400000.0,516.6672,774.192749
187860,2024-12-01,SENGKANG,3 ROOM,353B,ANCHORVALE LANE,16 TO 18,68.0,Model A,1830-09-01,570000.0,731.9452,778.746824


In [22]:
if for_editing.shape[0] > 0:
    for i in for_editing.index:
        search_value = (
            for_editing.loc[i, "block"] + " " + for_editing.loc[i, "street_name"]
        )
        print(f"{i}: {search_value}")
        response = requests.get(
            f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        )
        while response.status_code != 200:
            time.sleep(1)
            print("not 200")
            response = requests.get(
                f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
            )

        json_data = response.json()
        for j in json_data["results"]:
            print(j)
        time.sleep(0.5)
        user_input = input("Select the correct address: ")
        k = int(user_input) - 1
        print(k)

        if k == -2:
            for_editing.loc[i, "found"] = math.nan
            for_editing.loc[i, "search_val"] = math.nan
            for_editing.loc[i, "blk_no"] = math.nan
            for_editing.loc[i, "road_name"] = math.nan
            for_editing.loc[i, "building"] = math.nan
            for_editing.loc[i, "address"] = math.nan
            for_editing.loc[i, "postal"] = math.nan
            for_editing.loc[i, "x"] = math.nan
            for_editing.loc[i, "y"] = math.nan
            for_editing.loc[i, "latitude"] = math.nan
            for_editing.loc[i, "longitude"] = math.nan
        else:
            for_editing.loc[i, "found"] = 1
            for_editing.loc[i, "search_val"] = json_data["results"][k]["SEARCHVAL"]
            for_editing.loc[i, "blk_no"] = json_data["results"][k]["BLK_NO"]
            for_editing.loc[i, "road_name"] = json_data["results"][k]["ROAD_NAME"]
            for_editing.loc[i, "building"] = json_data["results"][k]["BUILDING"]
            for_editing.loc[i, "address"] = json_data["results"][k]["ADDRESS"]
            for_editing.loc[i, "postal"] = json_data["results"][k]["POSTAL"]
            for_editing.loc[i, "x"] = json_data["results"][k]["X"]
            for_editing.loc[i, "y"] = json_data["results"][k]["Y"]
            for_editing.loc[i, "latitude"] = json_data["results"][k]["LATITUDE"]
            for_editing.loc[i, "longitude"] = json_data["results"][k]["LONGITUDE"]

186180: 79 DAWSON RD
{'SEARCHVAL': 'DAWSON VISTA', 'BLK_NO': '79', 'ROAD_NAME': 'DAWSON ROAD', 'BUILDING': 'DAWSON VISTA', 'ADDRESS': '79 DAWSON ROAD DAWSON VISTA SINGAPORE 141079', 'POSTAL': '141079', 'X': '25441.1974788317', 'Y': '30774.8071813639', 'LATITUDE': '1.29459083160081', 'LONGITUDE': '103.810326636243'}
0
187186: 126C CANBERRA ST
{'SEARCHVAL': 'EASTDELTA @ CANBERRA', 'BLK_NO': '126C', 'ROAD_NAME': 'CANBERRA STREET', 'BUILDING': 'EASTDELTA @ CANBERRA', 'ADDRESS': '126C CANBERRA STREET EASTDELTA @ CANBERRA SINGAPORE 753126', 'POSTAL': '753126', 'X': '28183.1818723846', 'Y': '47532.3992327806', 'LATITUDE': '1.44614061239177', 'LONGITUDE': '103.834964649867'}
0
187598: 102A CANBERRA ST
{'SEARCHVAL': 'EASTCREEK @ CANBERRA', 'BLK_NO': '102A', 'ROAD_NAME': 'CANBERRA STREET', 'BUILDING': 'EASTCREEK @ CANBERRA', 'ADDRESS': '102A CANBERRA STREET EASTCREEK @ CANBERRA SINGAPORE 751102', 'POSTAL': '751102', 'X': '27878.5864955941', 'Y': '48091.1669760384', 'LATITUDE': '1.45119390519494'

In [23]:
for_editing_gdf = gpd.GeoDataFrame(
    for_editing, geometry=gpd.points_from_xy(for_editing["x"], for_editing["y"])
)

for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry
186180,2024-12-01,QUEENSTOWN,3 ROOM,79,DAWSON RD,22 TO 24,62.0,Model A,1831-01-01,766000.0,667.3618,1147.803186,1.0,DAWSON VISTA,79,DAWSON ROAD,DAWSON VISTA,79 DAWSON ROAD DAWSON VISTA SINGAPORE 141079,141079,25441.1974788317,30774.8071813639,1.29459083160081,103.810326636243,POINT (25441.197 30774.807)
187186,2024-12-01,SEMBAWANG,4 ROOM,126C,CANBERRA ST,07 TO 09,93.0,Model A,1831-01-01,695000.0,1001.0427,694.276078,1.0,EASTDELTA @ CANBERRA,126C,CANBERRA STREET,EASTDELTA @ CANBERRA,126C CANBERRA STREET EASTDELTA @ CANBERRA SING...,753126,28183.1818723846,47532.3992327806,1.44614061239177,103.834964649867,POINT (28183.182 47532.399)
187598,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,116.0,Improved,1830-12-01,755000.0,1248.6124,604.671233,1.0,EASTCREEK @ CANBERRA,102A,CANBERRA STREET,EASTCREEK @ CANBERRA,102A CANBERRA STREET EASTCREEK @ CANBERRA SING...,751102,27878.5864955941,48091.1669760384,1.45119390519494,103.832227554636,POINT (27878.586 48091.167)
187841,2024-12-01,SENGKANG,2 ROOM,456B,SENGKANG WEST RD,04 TO 06,48.0,2-room,1831-03-01,400000.0,516.6672,774.192749,1.0,FERNVALE WOODS,456B,SENGKANG WEST ROAD,FERNVALE WOODS,456B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792456,32072.0051863333,41558.6311787739,1.39211572540583,103.869908748735,POINT (32072.005 41558.631)
187860,2024-12-01,SENGKANG,3 ROOM,353B,ANCHORVALE LANE,16 TO 18,68.0,Model A,1830-09-01,570000.0,731.9452,778.746824,1.0,ANCHORVALE PLAINS,353B,ANCHORVALE LANE,ANCHORVALE PLAINS,353B ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,542353,33846.8139346438,41805.4785418436,1.39434782021906,103.88585684721,POINT (33846.814 41805.479)
191062,2024-12-01,TAMPINES,4 ROOM,608B,TAMPINES NTH DR 1,10 TO 12,93.0,Model A,1831-01-01,775000.0,1001.0427,774.192749,1.0,TAMPINES GREENWEAVE,608B,TAMPINES NORTH DRIVE 1,TAMPINES GREENWEAVE,608B TAMPINES NORTH DRIVE 1 TAMPINES GREENWEAV...,522608,39422.3514540456,38353.276432056,1.36312572875358,103.935956103971,POINT (39422.351 38353.276)
191072,2024-12-01,TAMPINES,4 ROOM,613B,TAMPINES NTH DR 1,07 TO 09,93.0,Model A,1831-02-01,750000.0,1001.0427,749.218790,1.0,TAMPINES GREENVIEW,613B,TAMPINES NORTH DRIVE 1,TAMPINES GREENVIEW,613B TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,522613,39428.1491047191,38680.0813656524,1.366081228979,103.936008325034,POINT (39428.149 38680.081)
192326,2024-12-01,TOA PAYOH,3 ROOM,115C,ALKAFF CRES,13 TO 15,71.0,Model A,1830-12-01,900000.0,764.2369,1177.645309,1.0,ALKAFF LAKEVIEW,115C,ALKAFF CRESCENT,ALKAFF LAKEVIEW,115C ALKAFF CRESCENT ALKAFF LAKEVIEW SINGAPORE...,343115,32403.5045328625,35489.4825334083,1.33722847497236,103.872886634076,POINT (32403.505 35489.483)


#### 1c. Appending MRT information

In [25]:
mrt_file_loc = "../datasets/mrt_lrt_stations.csv"
mrt_df = pd.read_csv(mrt_file_loc, parse_dates=["opening"], index_col=0)
mrt_gdf = gpd.GeoDataFrame(mrt_df, geometry=gpd.points_from_xy(mrt_df["x"], mrt_df["y"]))

mrt_gdf.head(1)

,code,station_name,line,color,opening,type,blk_no,road_name,building,address,postal,x,y,latitude,longitude,planning_area_ura,region_ura,geometry
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.057052,35038.96887,1.333153,103.742286,JURONG EAST,WEST REGION,POINT (17869.057 35038.969)


In [26]:
def find_closest_station(row, mrt_gdf):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # mrt_gdf["OPENING_DATE"] = mrt_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = mrt_gdf[mrt_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    """Remove comments for this line if you want the closest MRT station today"""
    distances = mrt_gdf.distance(row["geometry"])

    closest_station_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_station_name = mrt_gdf.loc[closest_station_index, "station_name"]
    closest_transport_type = mrt_gdf.loc[closest_station_index, "type"]
    closest_mrt_color = mrt_gdf.loc[closest_station_index, "color"]

    # distance to cbd
    raffles_place_index = mrt_gdf.query("station_name == 'Raffles Place'").index[0]
    distance_to_cbd = mrt_gdf.loc[raffles_place_index, "geometry"].distance(
        row["geometry"]
    )

    return pd.Series(
        [
            closest_station_name,
            shortest_distance,
            closest_transport_type,
            closest_mrt_color,
            distance_to_cbd,
        ],
        index=[
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ],
    )


if for_editing_gdf.shape[0] > 0:
    for_editing_gdf[
        [
            "closest_mrt_station",
            "distance_to_mrt_meters",
            "transport_type",
            "line_color",
            "distance_to_cbd",
        ]
    ] = for_editing_gdf.apply(find_closest_station, mrt_gdf=mrt_gdf, axis=1)

In [27]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd
186180,2024-12-01,QUEENSTOWN,3 ROOM,79,DAWSON RD,22 TO 24,62.0,Model A,1831-01-01,766000.0,667.3618,1147.803186,1.0,DAWSON VISTA,79,DAWSON ROAD,DAWSON VISTA,79 DAWSON ROAD DAWSON VISTA SINGAPORE 141079,141079,25441.1974788317,30774.8071813639,1.29459083160081,103.810326636243,POINT (25441.197 30774.807),Queenstown,472.950750,MRT,Green,4721.972907
187186,2024-12-01,SEMBAWANG,4 ROOM,126C,CANBERRA ST,07 TO 09,93.0,Model A,1831-01-01,695000.0,1001.0427,694.276078,1.0,EASTDELTA @ CANBERRA,126C,CANBERRA STREET,EASTDELTA @ CANBERRA,126C CANBERRA STREET EASTDELTA @ CANBERRA SING...,753126,28183.1818723846,47532.3992327806,1.44614061239177,103.834964649867,POINT (28183.182 47532.399),Canberra,676.530884,MRT,Red,18008.622149
187598,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,116.0,Improved,1830-12-01,755000.0,1248.6124,604.671233,1.0,EASTCREEK @ CANBERRA,102A,CANBERRA STREET,EASTCREEK @ CANBERRA,102A CANBERRA STREET EASTCREEK @ CANBERRA SING...,751102,27878.5864955941,48091.1669760384,1.45119390519494,103.832227554636,POINT (27878.586 48091.167),Canberra,940.521144,MRT,Red,18597.159854
187841,2024-12-01,SENGKANG,2 ROOM,456B,SENGKANG WEST RD,04 TO 06,48.0,2-room,1831-03-01,400000.0,516.6672,774.192749,1.0,FERNVALE WOODS,456B,SENGKANG WEST ROAD,FERNVALE WOODS,456B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792456,32072.0051863333,41558.6311787739,1.39211572540583,103.869908748735,POINT (32072.005 41558.631),Fernvale,712.675628,LRT,Grey,12116.188103
187860,2024-12-01,SENGKANG,3 ROOM,353B,ANCHORVALE LANE,16 TO 18,68.0,Model A,1830-09-01,570000.0,731.9452,778.746824,1.0,ANCHORVALE PLAINS,353B,ANCHORVALE LANE,ANCHORVALE PLAINS,353B ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,542353,33846.8139346438,41805.4785418436,1.39434782021906,103.88585684721,POINT (33846.814 41805.479),Farmway,494.603904,LRT,Grey,12774.774130
191062,2024-12-01,TAMPINES,4 ROOM,608B,TAMPINES NTH DR 1,10 TO 12,93.0,Model A,1831-01-01,775000.0,1001.0427,774.192749,1.0,TAMPINES GREENWEAVE,608B,TAMPINES NORTH DRIVE 1,TAMPINES GREENWEAVE,608B TAMPINES NORTH DRIVE 1 TAMPINES GREENWEAV...,522608,39422.3514540456,38353.276432056,1.36312572875358,103.935956103971,POINT (39422.351 38353.276),Tampines,1180.648200,MRT,Blue,12834.774884
191072,2024-12-01,TAMPINES,4 ROOM,613B,TAMPINES NTH DR 1,07 TO 09,93.0,Model A,1831-02-01,750000.0,1001.0427,749.218790,1.0,TAMPINES GREENVIEW,613B,TAMPINES NORTH DRIVE 1,TAMPINES GREENVIEW,613B TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,522613,39428.1491047191,38680.0813656524,1.366081228979,103.936008325034,POINT (39428.149 38680.081),Tampines,1438.068071,MRT,Blue,13063.576466
192326,2024-12-01,TOA PAYOH,3 ROOM,115C,ALKAFF CRES,13 TO 15,71.0,Model A,1830-12-01,900000.0,764.2369,1177.645309,1.0,ALKAFF LAKEVIEW,115C,ALKAFF CRESCENT,ALKAFF LAKEVIEW,115C ALKAFF CRESCENT ALKAFF LAKEVIEW SINGAPORE...,343115,32403.5045328625,35489.4825334083,1.33722847497236,103.872886634076,POINT (32403.505 35489.483),Woodleigh,316.274896,MRT,Purple,6337.496166


#### 1d. Appending Closest School information

In [28]:
school_file_loc = "../datasets/schools_for_plotly.csv"
school_df = pd.read_csv(school_file_loc, index_col=0, dtype={"postal":"string"})
school_df["postal"] = school_df["postal"].astype("str").apply(lambda x: f"{x:0>6}")

# Convert the df into a gdf
school_gdf = gpd.GeoDataFrame(school_df, geometry=gpd.points_from_xy(school_df["x"], school_df["y"]))

In [29]:
def find_closest_school(row, school_gdf, level="PRIMARY"):
    """Remove comments for the following 3 lines if you want the closest MRT at the time of transaction"""
    # school_gdf["OPENING_DATE"] = school_gdf["OPENING"].dt.to_period("M").dt.to_timestamp()
    # mrt_stations_filtered = school_gdf[school_gdf["OPENING"] < row["month"]]
    # distances = mrt_stations_filtered.distance(row["geometry"])

    school_gdf_filtered = school_gdf.query("mainlevel_code == @level")
    """Remove comments for this line if you want the closest MRT station today"""
    distances = school_gdf_filtered.distance(row["geometry"])

    closest_school_index = distances.idxmin()
    shortest_distance = distances.min()

    closest_school = school_gdf_filtered.loc[closest_school_index, "school_name"]

    return pd.Series(
        [
            closest_school,
            shortest_distance,
        ],
        index=[
            "closest_pri_school",
            "distance_to_pri_school_meters",
        ],
    )

for_editing_gdf[["closest_pri_school", "distance_to_pri_school_meters"]] = (
    for_editing_gdf.apply(find_closest_school, school_gdf=school_gdf, axis=1)
)

In [30]:
for_editing_gdf

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,floor_area_sqft,price_per_sqft,found,search_val,blk_no,road_name,building,address,postal,x,y,latitude,longitude,geometry,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
186180,2024-12-01,QUEENSTOWN,3 ROOM,79,DAWSON RD,22 TO 24,62.0,Model A,1831-01-01,766000.0,667.3618,1147.803186,1.0,DAWSON VISTA,79,DAWSON ROAD,DAWSON VISTA,79 DAWSON ROAD DAWSON VISTA SINGAPORE 141079,141079,25441.1974788317,30774.8071813639,1.29459083160081,103.810326636243,POINT (25441.197 30774.807),Queenstown,472.950750,MRT,Green,4721.972907,QUEENSTOWN PRIMARY SCHOOL,328.453068
187186,2024-12-01,SEMBAWANG,4 ROOM,126C,CANBERRA ST,07 TO 09,93.0,Model A,1831-01-01,695000.0,1001.0427,694.276078,1.0,EASTDELTA @ CANBERRA,126C,CANBERRA STREET,EASTDELTA @ CANBERRA,126C CANBERRA STREET EASTDELTA @ CANBERRA SING...,753126,28183.1818723846,47532.3992327806,1.44614061239177,103.834964649867,POINT (28183.182 47532.399),Canberra,676.530884,MRT,Red,18008.622149,CHONGFU SCHOOL,983.450206
187598,2024-12-01,SEMBAWANG,5 ROOM,102A,CANBERRA ST,04 TO 06,116.0,Improved,1830-12-01,755000.0,1248.6124,604.671233,1.0,EASTCREEK @ CANBERRA,102A,CANBERRA STREET,EASTCREEK @ CANBERRA,102A CANBERRA STREET EASTCREEK @ CANBERRA SING...,751102,27878.5864955941,48091.1669760384,1.45119390519494,103.832227554636,POINT (27878.586 48091.167),Canberra,940.521144,MRT,Red,18597.159854,WELLINGTON PRIMARY SCHOOL,1107.280375
187841,2024-12-01,SENGKANG,2 ROOM,456B,SENGKANG WEST RD,04 TO 06,48.0,2-room,1831-03-01,400000.0,516.6672,774.192749,1.0,FERNVALE WOODS,456B,SENGKANG WEST ROAD,FERNVALE WOODS,456B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792456,32072.0051863333,41558.6311787739,1.39211572540583,103.869908748735,POINT (32072.005 41558.631),Fernvale,712.675628,LRT,Grey,12116.188103,FERNVALE PRIMARY SCHOOL,543.239487
187860,2024-12-01,SENGKANG,3 ROOM,353B,ANCHORVALE LANE,16 TO 18,68.0,Model A,1830-09-01,570000.0,731.9452,778.746824,1.0,ANCHORVALE PLAINS,353B,ANCHORVALE LANE,ANCHORVALE PLAINS,353B ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,542353,33846.8139346438,41805.4785418436,1.39434782021906,103.88585684721,POINT (33846.814 41805.479),Farmway,494.603904,LRT,Grey,12774.774130,SPRINGDALE PRIMARY SCHOOL,454.267545
191062,2024-12-01,TAMPINES,4 ROOM,608B,TAMPINES NTH DR 1,10 TO 12,93.0,Model A,1831-01-01,775000.0,1001.0427,774.192749,1.0,TAMPINES GREENWEAVE,608B,TAMPINES NORTH DRIVE 1,TAMPINES GREENWEAVE,608B TAMPINES NORTH DRIVE 1 TAMPINES GREENWEAV...,522608,39422.3514540456,38353.276432056,1.36312572875358,103.935956103971,POINT (39422.351 38353.276),Tampines,1180.648200,MRT,Blue,12834.774884,ANGSANA PRIMARY SCHOOL,336.389862
191072,2024-12-01,TAMPINES,4 ROOM,613B,TAMPINES NTH DR 1,07 TO 09,93.0,Model A,1831-02-01,750000.0,1001.0427,749.218790,1.0,TAMPINES GREENVIEW,613B,TAMPINES NORTH DRIVE 1,TAMPINES GREENVIEW,613B TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,522613,39428.1491047191,38680.0813656524,1.366081228979,103.936008325034,POINT (39428.149 38680.081),Tampines,1438.068071,MRT,Blue,13063.576466,ANGSANA PRIMARY SCHOOL,425.944107
192326,2024-12-01,TOA PAYOH,3 ROOM,115C,ALKAFF CRES,13 TO 15,71.0,Model A,1830-12-01,900000.0,764.2369,1177.645309,1.0,ALKAFF LAKEVIEW,115C,ALKAFF CRESCENT,ALKAFF LAKEVIEW,115C ALKAFF CRESCENT ALKAFF LAKEVIEW SINGAPORE...,343115,32403.5045328625,35489.4825334083,1.33722847497236,103.872886634076,POINT (32403.505 35489.483),Woodleigh,316.274896,MRT,Purple,6337.496166,CEDAR PRIMARY SCHOOL,354.245245


#### 1e. Appending URA planning area and regions

In [31]:
geo_file_loc = "../datasets/sg_map/mp2014/MP14_PLNG_AREA_NO_SEA_PL.shp"
planning_areas_gdf = gpd.read_file(geo_file_loc)

planning_areas_gdf.head()

,OBJECTID,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,ANG MO KIO,AM,N,NORTH-EAST REGION,NER,E5CBDDE0C2113055,2016-05-11,28976.8763,40229.1238,17494.240190,1.394138e+07,"POLYGON ((30658.500 42047.527, 30679.195 42020..."
1,2,BEDOK,BD,N,EAST REGION,ER,1719251260799DF6,2016-05-11,38582.6650,34032.0961,21872.798962,2.173319e+07,"POLYGON ((38974.269 36138.243, 39371.471 35747..."
2,3,BISHAN,BS,N,CENTRAL REGION,CR,BA616285F402846F,2016-05-11,28789.7630,37450.8865,13517.121556,7.618921e+06,"POLYGON ((29772.191 38311.805, 29784.826 38304..."
3,4,BOON LAY,BL,N,WEST REGION,WR,A3DC87118B43CDED,2016-05-11,13410.3824,33008.9884,18528.467448,8.279408e+06,"POLYGON ((12861.383 32207.492, 12860.555 32208..."
4,5,BUKIT BATOK,BK,N,WEST REGION,WR,FB44C870B04B7F57,2016-05-11,19255.4150,37527.6527,15234.223423,1.113326e+07,"POLYGON ((20294.455 39114.528, 20334.318 39054..."


In [32]:
for_editing_gdf = for_editing_gdf.sjoin(planning_areas_gdf[["PLN_AREA_N", "REGION_N", "geometry"]], how='left')

/Users/tim/miniforge3/envs/for_ml/lib/python3.10/site-packages/geopandas/geodataframe.py:2189: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: PROJCS["SVY21",GEOGCS["SVY21[WGS84]",DATUM["WGS_19 ...

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


In [33]:
for_editing_gdf = for_editing_gdf.rename(columns={"PLN_AREA_N":"planning_area_ura", "REGION_N":"region_ura"})

In [34]:
for_editing = for_editing_gdf[
    [
        "town",
        "block",
        "road_name",
        "blk_no",
        "street_name",
        "building",
        "postal",
        "address",
        "lease_commence_date",
        "planning_area_ura",
        "region_ura",
        "x",
        "y",
        "latitude",
        "longitude",
        "closest_mrt_station",
        "distance_to_mrt_meters",
        "transport_type",
        "line_color",
        "distance_to_cbd",
        "closest_pri_school",
        "distance_to_pri_school_meters",
    ]
]

In [35]:
for_editing

,town,block,road_name,blk_no,street_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
186180,QUEENSTOWN,79,DAWSON ROAD,79,DAWSON RD,DAWSON VISTA,141079,79 DAWSON ROAD DAWSON VISTA SINGAPORE 141079,1831-01-01,QUEENSTOWN,CENTRAL REGION,25441.1974788317,30774.8071813639,1.29459083160081,103.810326636243,Queenstown,472.950750,MRT,Green,4721.972907,QUEENSTOWN PRIMARY SCHOOL,328.453068
187186,SEMBAWANG,126C,CANBERRA STREET,126C,CANBERRA ST,EASTDELTA @ CANBERRA,753126,126C CANBERRA STREET EASTDELTA @ CANBERRA SING...,1831-01-01,SEMBAWANG,NORTH REGION,28183.1818723846,47532.3992327806,1.44614061239177,103.834964649867,Canberra,676.530884,MRT,Red,18008.622149,CHONGFU SCHOOL,983.450206
187598,SEMBAWANG,102A,CANBERRA STREET,102A,CANBERRA ST,EASTCREEK @ CANBERRA,751102,102A CANBERRA STREET EASTCREEK @ CANBERRA SING...,1830-12-01,SEMBAWANG,NORTH REGION,27878.5864955941,48091.1669760384,1.45119390519494,103.832227554636,Canberra,940.521144,MRT,Red,18597.159854,WELLINGTON PRIMARY SCHOOL,1107.280375
187841,SENGKANG,456B,SENGKANG WEST ROAD,456B,SENGKANG WEST RD,FERNVALE WOODS,792456,456B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,1831-03-01,SENGKANG,NORTH-EAST REGION,32072.0051863333,41558.6311787739,1.39211572540583,103.869908748735,Fernvale,712.675628,LRT,Grey,12116.188103,FERNVALE PRIMARY SCHOOL,543.239487
187860,SENGKANG,353B,ANCHORVALE LANE,353B,ANCHORVALE LANE,ANCHORVALE PLAINS,542353,353B ANCHORVALE LANE ANCHORVALE PLAINS SINGAPO...,1830-09-01,SENGKANG,NORTH-EAST REGION,33846.8139346438,41805.4785418436,1.39434782021906,103.88585684721,Farmway,494.603904,LRT,Grey,12774.774130,SPRINGDALE PRIMARY SCHOOL,454.267545
191062,TAMPINES,608B,TAMPINES NORTH DRIVE 1,608B,TAMPINES NTH DR 1,TAMPINES GREENWEAVE,522608,608B TAMPINES NORTH DRIVE 1 TAMPINES GREENWEAV...,1831-01-01,TAMPINES,EAST REGION,39422.3514540456,38353.276432056,1.36312572875358,103.935956103971,Tampines,1180.648200,MRT,Blue,12834.774884,ANGSANA PRIMARY SCHOOL,336.389862
191072,TAMPINES,613B,TAMPINES NORTH DRIVE 1,613B,TAMPINES NTH DR 1,TAMPINES GREENVIEW,522613,613B TAMPINES NORTH DRIVE 1 TAMPINES GREENVIEW...,1831-02-01,TAMPINES,EAST REGION,39428.1491047191,38680.0813656524,1.366081228979,103.936008325034,Tampines,1438.068071,MRT,Blue,13063.576466,ANGSANA PRIMARY SCHOOL,425.944107
192326,TOA PAYOH,115C,ALKAFF CRESCENT,115C,ALKAFF CRES,ALKAFF LAKEVIEW,343115,115C ALKAFF CRESCENT ALKAFF LAKEVIEW SINGAPORE...,1830-12-01,TOA PAYOH,CENTRAL REGION,32403.5045328625,35489.4825334083,1.33722847497236,103.872886634076,Woodleigh,316.274896,MRT,Purple,6337.496166,CEDAR PRIMARY SCHOOL,354.245245


In [36]:
new_masterlist_df = pd.concat([masterlist_df, for_editing], axis=0)
new_masterlist_df = new_masterlist_df.sort_values(by=["town", "street_name", "block"]).reset_index(drop=True)

In [37]:
new_masterlist_df.shape

(9655, 22)

In [38]:
# new_masterlist_df.to_csv(masterlist_file_loc)

### 2. Append additional information to the raw dataset

#### 2a: Reopen Masterlist

In [39]:
masterlist_file_loc = "../datasets/hdb_resale_flat_address_masterlist.csv"
# masterlist_file_loc = "./temp_masterlist.csv"
masterlist_df = pd.read_csv(
    masterlist_file_loc,
    index_col=0,
    dtype={"postal": "object"},
    parse_dates=["lease_commence_date"],
)

masterlist_df.head(1)

,town,block,street_name,blk_no,road_name,building,postal,address,lease_commence_date,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,ANG MO KIO,205,ANG MO KIO AVE 1,205,ANG MO KIO AVENUE 1,NIL,560205,205 ANG MO KIO AVENUE 1 SINGAPORE 560205,1977-01-01,ANG MO KIO,NORTH-EAST REGION,29142.244275,38774.891527,1.366941,103.843582,Ang Mo Kio,742.768808,MRT,Red,9199.172507,ANG MO KIO PRIMARY SCHOOL,512.545254


In [40]:
masterlist_df.shape

(9655, 22)

#### 2b: Perform a left merge between the new HDB dataset and the address masterlist

In [41]:
df_new = df_raw.merge(
    masterlist_df,
    how="left",
    on=["town", "block", "street_name"],
    suffixes=["", "_r"],
    indicator=True,
)

In [42]:
df_new.query("_merge != 'both'")

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,_merge


In [43]:
df_new = df_new.drop("_merge", axis=1)

In [44]:
file_destination = file_loc_raw.replace("raw", "coords_mrt")
file_destination
df_new.to_csv(file_destination)

#### The following is used to call the Onemap API and populate coordinate details (deprecated, use the above masterlist instead)

In [ ]:
# # Obtain geospatial coordinates with the Onemap API

# def get_coordinates_for_each_row(row):
#     search_value = row["block"] + " " + row["street_name"]
#     # print(search_value)

#     response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")
#     while response.status_code != 200:
#         time.sleep(1)
#         print("not 200")
#         response = requests.get(f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={search_value}&returnGeom=Y&getAddrDetails=Y&pageNum=1")

#     data = response.json()

#     row_data = data['results'][0]
#     searchval = row_data["SEARCHVAL"]
#     address = row_data["ADDRESS"]
#     postal = row_data["POSTAL"]
#     x = row_data["X"]
#     y = row_data["Y"]
#     latitude = row_data["LATITUDE"]
#     longitude = row_data["LONGITUDE"]

#     return pd.Series([address, postal, x, y, latitude, longitude], 
#                      index=["SEARCHVAL", "ADDRESS", "POSTAL", "X", "Y", "LATITUDE", "LONGITUDE"])

In [ ]:
# Appending additional columns to the dataframe


# NOTE: SHOULDNT APPEND
# for_addition["year"] = for_addition["month"].dt.year
# for_addition["lease_years"] = for_addition["remaining_lease"].str.split(" ").apply(lambda x: int(x[0]))
# bins = pd.IntervalIndex.from_tuples(
#     [(40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]
# )
# for_addition["lease_cat"] = pd.cut(for_addition["lease_years"], bins)
# for_addition = for_addition.drop("index", axis=1)

#### Checking datasets

In [45]:
file_loc_cleaned = "../datasets/resale_hdb_price_coords_mrt_13jan.csv"
df_for_kaggle = pd.read_csv(
    file_loc_cleaned,
    parse_dates=["month", "lease_commence_date"],
    index_col=0,
    dtype={"x": "float64", "y": "float64", "postal": "object"},
    low_memory=False,
)

In [46]:
df_for_kaggle[df_for_kaggle.isna().any(axis=1)]
# df_for_kaggle.shape

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters


In [47]:
# df_for_kaggle = df_for_kaggle.drop([ "block", "street_name"], axis=1)
df_for_kaggle = df_for_kaggle.drop(["town", "block", "street_name"], axis=1)

In [48]:
today_date = dt.date.today().strftime("%Y-%d%b").lower()
kaggle_file_loc_updated_name = f"../datasets/resale_hdb_price_for_kaggle_{today_date}.csv"
df_for_kaggle.to_csv(kaggle_file_loc_updated_name)

In [49]:
df_for_kaggle

,month,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,floor_area_sqft,price_per_sqft,blk_no,road_name,building,postal,address,lease_commence_date_r,planning_area_ura,region_ura,x,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,2017-01-01,2 ROOM,10 TO 12,44.0,Improved,1979-01-01,61 years 04 months,232000.0,473.6116,489.852867,406,ANG MO KIO AVENUE 10,NIL,560406,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,1979-05-01,ANG MO KIO,NORTH-EAST REGION,30288.234663,38229.067463,1.362005,103.853880,Ang Mo Kio,999.941618,MRT,Red,8615.656983,TOWNSVILLE PRIMARY SCHOOL,218.125254
1,2017-01-01,3 ROOM,01 TO 03,67.0,New Generation,1978-01-01,60 years 07 months,250000.0,721.1813,346.653470,108,ANG MO KIO AVENUE 4,KEBUN BARU HEIGHTS,560108,108 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SIN...,1978-08-01,ANG MO KIO,NORTH-EAST REGION,28543.458747,39220.009892,1.370966,103.838202,Mayflower,189.980291,MRT,Brown,9715.131951,ANG MO KIO PRIMARY SCHOOL,241.572335
2,2017-01-01,3 ROOM,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,262000.0,721.1813,363.292836,602,ANG MO KIO AVENUE 5,YIO CHU KANG GREEN,560602,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1980-06-01,ANG MO KIO,NORTH-EAST REGION,28228.099954,40297.283149,1.380709,103.835368,Lentor,532.154773,MRT,Brown,10828.819556,ANDERSON PRIMARY SCHOOL,777.155378
3,2017-01-01,3 ROOM,04 TO 06,68.0,New Generation,1980-01-01,62 years 01 month,265000.0,731.9452,362.048962,465,ANG MO KIO AVENUE 10,TECK GHEE HORIZON,560465,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,1980-02-01,ANG MO KIO,NORTH-EAST REGION,30657.824693,38693.098657,1.366201,103.857201,Ang Mo Kio,945.371842,MRT,Red,9097.929095,TECK GHEE PRIMARY SCHOOL,698.165530
4,2017-01-01,3 ROOM,01 TO 03,67.0,New Generation,1980-01-01,62 years 05 months,265000.0,721.1813,367.452678,601,ANG MO KIO AVENUE 5,YIO CHU KANG GREEN,560601,601 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,1980-06-01,ANG MO KIO,NORTH-EAST REGION,28201.782245,40334.052030,1.381041,103.835132,Lentor,498.418205,MRT,Brown,10869.453109,ANDERSON PRIMARY SCHOOL,782.553222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197003,2024-12-01,EXECUTIVE,07 TO 09,142.0,Apartment,1988-01-01,62 years 03 months,950000.0,1528.4738,621.535024,834,YISHUN STREET 81,NIL,760834,834 YISHUN STREET 81 SINGAPORE 760834,1988-01-01,YISHUN,NORTH REGION,28075.777355,44098.765290,1.415088,103.834000,Khatib,278.011616,MRT,Red,14610.975312,PEIYING PRIMARY SCHOOL,463.909479
197004,2024-12-01,EXECUTIVE,07 TO 09,146.0,Maisonette,1988-01-01,62 years 03 months,990000.0,1571.5294,629.959580,828,YISHUN STREET 81,NIL,760828,828 YISHUN STREET 81 SINGAPORE 760828,1988-01-01,YISHUN,NORTH REGION,27953.606541,44110.138999,1.415191,103.832902,Khatib,242.585206,MRT,Red,14638.986228,PEIYING PRIMARY SCHOOL,357.013909
197005,2024-02-01,MULTI-GENERATION,04 TO 06,164.0,Multi Generation,1987-01-01,62 years 11 months,998000.0,1765.2796,565.349534,666,YISHUN AVENUE 4,NIL,760666,666 YISHUN AVENUE 4 SINGAPORE 760666,1987-12-01,YISHUN,NORTH REGION,28806.756365,44531.159086,1.418998,103.840568,Khatib,863.123975,MRT,Red,14962.748464,NORTHLAND PRIMARY SCHOOL,221.649812
197006,2024-03-01,MULTI-GENERATION,10 TO 12,164.0,Multi Generation,1987-01-01,62 years 09 months,1200000.0,1765.2796,679.778999,666,YISHUN AVENUE 4,NIL,760666,666 YISHUN AVENUE 4 SINGAPORE 760666,1987-12-01,YISHUN,NORTH REGION,28806.756365,44531.159086,1.418998,103.840568,Khatib,863.123975,MRT,Red,14962.748464,NORTHLAND PRIMARY SCHOOL,221.649812
